In [ ]:
# !pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchaudio==0.13.1 --extra-index-url https://download.pytorch.org/whl/cu117 torchmetrics torchinfo
!pip install torcheval torchmetrics torchinfo --extra-index-url https://download.pytorch.org/whl/cu117

In [ ]:
!pip install --upgrade tensorboard 2.3

In [ ]:
# from this site https://github.com/KastoneX/AutoData-Analysis-and-Price-Prediction-of-Used-Cars

import sys
import io
import time
import random
import math
import os
import copy
from glob import glob
from tqdm import tqdm
from datetime import datetime


import os
from pathlib import Path
import shutil

import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc

import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

from sklearn.preprocessing import LabelEncoder,OneHotEncoder,MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, f1_score,accuracy_score,recall_score,roc_auc_score,roc_curve, confusion_matrix

import torch
import torch.nn as nn
import torch.optim as optim
from torchinfo import summary
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, TensorDataset, ConcatDataset
import torchvision
from torchvision import datasets, models, transforms
from torchmetrics.functional.classification import auroc # auc
from collections import defaultdict
from torch.utils.tensorboard import SummaryWriter
from torcheval.metrics.functional import binary_f1_score, binary_accuracy, binary_precision
from torcheval.metrics.functional.classification import binary_recall

# from tensorboard.plugins

%matplotlib inline

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

Colour_Palette = ['#01BEFE', '#FF7D00', '#FFDD00', '#FF006D', '#ADFF02', '#8F00FF']
sns.set_palette(sns.color_palette(Colour_Palette))

tqdm.pandas()

import warnings
warnings.simplefilter('ignore')


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
if torch.cuda.is_available():
  print(torch.cuda.get_device_name())


In [ ]:
DRIVE_MOUNT_POINT = '/content/drive'

from google.colab import drive

drive.mount(DRIVE_MOUNT_POINT)
print('Data source import complete.')

EXPR_PATH = os.environ.get('EXPR_RESULT_PATH')
EXPR_PATH_IMAGES = os.path.join(EXPR_PATH,'ExperimentResultImages')

dotenv.load_dotenv('/content/drive/MyDrive/.env')


In [ ]:
util_script_path = os.environ.get('UTIL_SCRIPT_PATH')
if util_script_path is not None:
  sys.path.insert(0, util_script_path)
else:
  print("Warning: UTIL_SCRIPT_PATH environment variable is not set.")

In [ ]:

from stress_models import  LSTMModel, AlexNetModifiedVersion, ResNet18ModifiedVersion
from preprocess_and_train_datasets import get_samples_as_dataloader, get_classification_distribution



In [ ]:
!ls -ltr '{util_script_path}' | grep 'LSTM' | tail -n 20

In [ ]:
!ls -ltr '{util_script_path}' | grep 'AlexNet' | tail -n 30

In [ ]:
!ls -ltr '{EXPR_PATH}' | grep 'LSTM_Stress' | tail -n 20

In [ ]:
!ls -ltr '{EXPR_PATH}' | grep 'LSTM_V3_Stress' | tail -n 20

In [ ]:
!ls -ltr '{EXPR_PATH}' | grep 'LSTM_V4_Stress' | tail -n 20

In [ ]:
!ls -ltr '{EXPR_PATH}' | grep 'AlexNet' | tail -n 20
# !ls -ltr '/content/drive/MyDrive/BitirmeProje/BitirmeProje_3/ExperimentResults/LSTM_Stress_Detection__Seperated_Augmentation_And_Non_Balanced_1_2_Datasets_For_DP_2-AP_1-FP_5-TP_15'

In [ ]:
import tensorflow as tf
import tensorboard as tb
from packaging import version

In [ ]:
import traceback
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator, TENSORS

DEFAULT_SIZE_GUIDANCE = {
        "compressedHistograms": 1,
        "images": 1,
        "scalars": 0,  # 0 means load all
        "histograms": 1,
    }

In [ ]:
def tflog2pandas(path, scalar_metric_name):
    # from other site

In [ ]:
def get_train_accuracy_values_as_df_from_log_path(log_path):

  mean_accuracy_train = []
  mean_accuracy_val = []

  kfold_range = int((len(os.listdir(log_path)) - 1) / 5)

  for i in range(kfold_range):

    log_dir = f'{log_path}/Accuracy - {(i + 1)}_val/acc'
    ratio_values = tflog2pandas(os.path.join(log_dir, os.listdir(log_dir)[0]), f'Accuracy - {(i + 1)}')
    if i > 0:
      if len(mean_accuracy_val[0]) == len(ratio_values):
        mean_accuracy_val.append(list(ratio_values))
    else:
      mean_accuracy_val.append(list(ratio_values))

    log_dir = f'{log_path}/Accuracy - {(i + 1)}_train/acc'
    ratio_values = tflog2pandas(os.path.join(log_dir, os.listdir(log_dir)[0]), f'Accuracy - {(i + 1)}')
    if i > 0:
      if len(mean_accuracy_val[0]) == len(ratio_values):
        mean_accuracy_train.append(list(ratio_values))
    else:
      mean_accuracy_train.append(list(ratio_values))


  main_log_df = pd.DataFrame()

  log_df = pd.DataFrame()
  log_df['Ratio'] = np.mean(mean_accuracy_val, axis=0)
  log_df['Phase'] = ['Validation' for ratio_idx in range(len(log_df['Ratio']))]
  log_df['Epoch'] = [epoch_idx + 1 for epoch_idx in range(len(log_df['Ratio']))]
  main_log_df = pd.concat([main_log_df, log_df])

  log_df = pd.DataFrame()
  log_df['Ratio'] = np.mean(mean_accuracy_train, axis=0)
  log_df['Phase'] = ['Train' for ratio_idx in range(len(log_df['Ratio']))]
  log_df['Epoch'] = [epoch_idx + 1 for epoch_idx in range(len(log_df['Ratio']))]
  main_log_df = pd.concat([main_log_df, log_df])

  return main_log_df


In [ ]:
LSTM_LOG_DIRS = [sample_file for sample_file in sorted(Path(EXPR_PATH).iterdir(), key=os.path.getmtime) if sample_file.is_dir() and sample_file.name.startswith('LSTM_Stress')]

ignored_index_list = [7, 13, 14]
# ignored_index_list = []

lstm_procedure_path_infos = {}

for log_dir_idx in range(1, 16):
  if log_dir_idx not in ignored_index_list:

    procedure_key = ''
    sample_log_dir = LSTM_LOG_DIRS[- log_dir_idx].name
    if 'Non_Balance' in sample_log_dir:
      procedure_key = 'NON_BLNC'
    else:
      procedure_key = 'BLNC'

    if 'AP_1' in sample_log_dir and '__Seperated_Augmentation' in sample_log_dir:
      procedure_key = f'{procedure_key}--ONL_ORG'
    elif 'AP_ALL' in sample_log_dir and '__Seperated_Augmentation' in sample_log_dir:
      procedure_key = f'{procedure_key}--ORG_AUG_SEP'
    else:
      procedure_key = f'{procedure_key}--ORG_AUG_MIX'

    if 'FP_2' in sample_log_dir:
      procedure_key = f'{procedure_key}--ALL'
    else:
      procedure_key = f'{procedure_key}--TD'

    print(sample_log_dir, procedure_key)
    lstm_procedure_path_infos[procedure_key] = sample_log_dir


print(lstm_procedure_path_infos)

In [ ]:

lstm_accuracy_df_infos = {}

for lstm_procedure_path_infos_key in lstm_procedure_path_infos.keys():
  kfold_logs_path = f'{EXPR_PATH}/{lstm_procedure_path_infos[lstm_procedure_path_infos_key]}/'

  sample_files = [sample_file.name for sample_file in sorted(Path(kfold_logs_path).iterdir(), key=os.path.getmtime) if sample_file.is_dir() and sample_file.name.startswith('run')]

  kfold_logs_path = os.path.join(kfold_logs_path, sample_files[-1])
  for __ in range(4):
    kfold_logs_path = os.path.join(kfold_logs_path, os.listdir(kfold_logs_path)[0])

  accuracy_df = get_train_accuracy_values_as_df_from_log_path(kfold_logs_path)
  lstm_accuracy_df_infos[lstm_procedure_path_infos_key] = accuracy_df.copy()
  print(lstm_procedure_path_infos_key, accuracy_df.shape)




In [ ]:
LSTM_V2_LOG_DIRS = [sample_file for sample_file in sorted(Path(EXPR_PATH).iterdir(), key=os.path.getmtime) if sample_file.is_dir() and sample_file.name.startswith('LSTM_V2_Stress')]

lstm_v2_procedure_path_infos = {}

for log_dir_idx in range(0, 12):

  procedure_key = ''
  sample_log_dir = LSTM_V2_LOG_DIRS[log_dir_idx].name
  if 'Non_Balance' in sample_log_dir:
    procedure_key = 'NON_BLNC'
  else:
    procedure_key = 'BLNC'

  if 'AP_1' in sample_log_dir and '__Seperated_Augmentation' in sample_log_dir:
    procedure_key = f'{procedure_key}--ONL_ORG'
  elif 'AP_ALL' in sample_log_dir and '__Seperated_Augmentation' in sample_log_dir:
    procedure_key = f'{procedure_key}--ORG_AUG_SEP'
  else:
    procedure_key = f'{procedure_key}--ORG_AUG_MIX'

  if 'FP_2' in sample_log_dir:
    procedure_key = f'{procedure_key}--ALL'
  else:
    procedure_key = f'{procedure_key}--TD'

  print(sample_log_dir, procedure_key)
  lstm_v2_procedure_path_infos[procedure_key] = sample_log_dir


print(lstm_v2_procedure_path_infos)



In [ ]:

lstm_v2_accuracy_df_infos = {}

for lstm_v2_procedure_path_infos_key in lstm_v2_procedure_path_infos.keys():
  kfold_logs_path = f'{EXPR_PATH}/{lstm_v2_procedure_path_infos[lstm_v2_procedure_path_infos_key]}/'

  sample_files = [sample_file.name for sample_file in sorted(Path(kfold_logs_path).iterdir(), key=os.path.getmtime) if sample_file.is_dir() and sample_file.name.startswith('run')]

  kfold_logs_path = os.path.join(kfold_logs_path, sample_files[-1])
  for __ in range(4):
    kfold_logs_path = os.path.join(kfold_logs_path, os.listdir(kfold_logs_path)[0])

  accuracy_df = get_train_accuracy_values_as_df_from_log_path(kfold_logs_path)
  lstm_v2_accuracy_df_infos[lstm_v2_procedure_path_infos_key] = accuracy_df.copy()
  print(lstm_v2_procedure_path_infos_key, accuracy_df.shape)




In [ ]:
LSTM_V3_LOG_DIRS = [sample_file for sample_file in sorted(Path(EXPR_PATH).iterdir(), key=os.path.getmtime) if sample_file.is_dir() and sample_file.name.startswith('LSTM_V3_Stress')]

ignored_index_list = []

lstm_v3_procedure_path_infos = {}

for log_dir_idx in range(len(LSTM_V3_LOG_DIRS)):
  if log_dir_idx not in ignored_index_list:

    procedure_key = ''
    sample_log_dir = LSTM_V3_LOG_DIRS[- log_dir_idx].name
    if 'Non_Balance' in sample_log_dir:
      procedure_key = 'NON_BLNC'
    else:
      procedure_key = 'BLNC'

    if 'AP_1' in sample_log_dir and '__Seperated_Augmentation' in sample_log_dir:
      procedure_key = f'{procedure_key}--ONL_ORG'
    elif 'AP_ALL' in sample_log_dir and '__Seperated_Augmentation' in sample_log_dir:
      procedure_key = f'{procedure_key}--ORG_AUG_SEP'
    else:
      procedure_key = f'{procedure_key}--ORG_AUG_MIX'

    if 'FP_2' in sample_log_dir:
      procedure_key = f'{procedure_key}--ALL'
    elif 'FP_5' in sample_log_dir:
      procedure_key = f'{procedure_key}--TD'
    else:
      procedure_key = f'{procedure_key}--ALL'

    if 'Raw' in sample_log_dir:
      procedure_key = f'{procedure_key}--RAW'
    else:
      procedure_key = f'{procedure_key}--NON_RAW'

    print(sample_log_dir, procedure_key)
    lstm_v3_procedure_path_infos[procedure_key] = sample_log_dir


print(lstm_v3_procedure_path_infos)

In [ ]:

lstm_v3_accuracy_df_infos = {}

for lstm_v3_procedure_path_infos_key in lstm_v3_procedure_path_infos.keys():
  kfold_logs_path = f'{EXPR_PATH}/{lstm_v3_procedure_path_infos[lstm_v3_procedure_path_infos_key]}/'

  sample_files = [sample_file.name for sample_file in sorted(Path(kfold_logs_path).iterdir(), key=os.path.getmtime) if sample_file.is_dir() and sample_file.name.startswith('run')]

  kfold_logs_path = os.path.join(kfold_logs_path, sample_files[-1])
  for __ in range(4):
    kfold_logs_path = os.path.join(kfold_logs_path, os.listdir(kfold_logs_path)[0])

  accuracy_df = get_train_accuracy_values_as_df_from_log_path(kfold_logs_path)
  lstm_v3_accuracy_df_infos[lstm_v3_procedure_path_infos_key] = accuracy_df.copy()
  print(lstm_v3_procedure_path_infos_key, accuracy_df.shape)




In [ ]:
LSTM_V4_LOG_DIRS = [sample_file for sample_file in sorted(Path(EXPR_PATH).iterdir(), key=os.path.getmtime) if sample_file.is_dir() and sample_file.name.startswith('LSTM_V4_Stress')]

ignored_index_list = []
# ignored_index_list = []

lstm_v4_procedure_path_infos = {}

for log_dir_idx in range(len(LSTM_V4_LOG_DIRS)):
  if log_dir_idx not in ignored_index_list:

    procedure_key = ''
    sample_log_dir = LSTM_V4_LOG_DIRS[- log_dir_idx].name
    if 'Non_Balance' in sample_log_dir:
      procedure_key = 'NON_BLNC'
    else:
      procedure_key = 'BLNC'

    if 'AP_1' in sample_log_dir and '__Seperated_Augmentation' in sample_log_dir:
      procedure_key = f'{procedure_key}--ONL_ORG'
    elif 'AP_ALL' in sample_log_dir and '__Seperated_Augmentation' in sample_log_dir:
      procedure_key = f'{procedure_key}--ORG_AUG_SEP'
    else:
      procedure_key = f'{procedure_key}--ORG_AUG_MIX'

    if 'FP_2' in sample_log_dir:
      procedure_key = f'{procedure_key}--ALL'
    else:
      procedure_key = f'{procedure_key}--TD'

    print(sample_log_dir, procedure_key)
    lstm_v4_procedure_path_infos[procedure_key] = sample_log_dir


print(lstm_v4_procedure_path_infos)

In [ ]:

lstm_v4_accuracy_df_infos = {}

for lstm_v4_procedure_path_infos_key in lstm_v4_procedure_path_infos.keys():
  kfold_logs_path = f'{EXPR_PATH}/{lstm_v4_procedure_path_infos[lstm_v4_procedure_path_infos_key]}/'

  sample_files = [sample_file.name for sample_file in sorted(Path(kfold_logs_path).iterdir(), key=os.path.getmtime) if sample_file.is_dir() and sample_file.name.startswith('run')]

  kfold_logs_path = os.path.join(kfold_logs_path, sample_files[-1])
  for __ in range(4):
    kfold_logs_path = os.path.join(kfold_logs_path, os.listdir(kfold_logs_path)[0])

  accuracy_df = get_train_accuracy_values_as_df_from_log_path(kfold_logs_path)
  lstm_v4_accuracy_df_infos[lstm_v4_procedure_path_infos_key] = accuracy_df.copy()
  print(lstm_v4_procedure_path_infos_key, accuracy_df.shape)




In [ ]:
ALEXNET_LOG_DIRS = [sample_file for sample_file in sorted(Path(EXPR_PATH).iterdir(), key=os.path.getmtime) if sample_file.is_dir() and sample_file.name.startswith('AlexNet')]

ignored_index_list = [8, 14, 15]
alexnet_procedure_path_infos = {}

for log_dir_idx in range(1, 16):
  if log_dir_idx not in ignored_index_list:

    procedure_key = ''
    sample_log_dir = ALEXNET_LOG_DIRS[- log_dir_idx].name
    if 'Non_Balance' in sample_log_dir:
      procedure_key = 'NON_BLNC'
    else:
      procedure_key = 'BLNC'

    if 'AP_1' in sample_log_dir and '__Seperated_Augmentation' in sample_log_dir:
      procedure_key = f'{procedure_key}--ONL_ORG'
    elif 'AP_ALL' in sample_log_dir and '__Seperated_Augmentation' in sample_log_dir:
      procedure_key = f'{procedure_key}--ORG_AUG_SEP'
    else:
      procedure_key = f'{procedure_key}--ORG_AUG_MIX'

    if 'FP_2' in sample_log_dir:
      procedure_key = f'{procedure_key}--ALL'
    else:
      procedure_key = f'{procedure_key}--TD'

    print(sample_log_dir, procedure_key)
    alexnet_procedure_path_infos[procedure_key] = sample_log_dir


print(alexnet_procedure_path_infos)

In [ ]:

alexnet_accuracy_df_infos = {}

for alexnet_procedure_path_infos_key in alexnet_procedure_path_infos.keys():
  kfold_logs_path = f'{EXPR_PATH}/{alexnet_procedure_path_infos[alexnet_procedure_path_infos_key]}/'

  sample_files = [sample_file.name for sample_file in sorted(Path(kfold_logs_path).iterdir(), key=os.path.getmtime) if sample_file.is_dir() and sample_file.name.startswith('run')]

  kfold_logs_path = os.path.join(kfold_logs_path, sample_files[-1])
  for __ in range(4):
    kfold_logs_path = os.path.join(kfold_logs_path, os.listdir(kfold_logs_path)[0])

  accuracy_df = get_train_accuracy_values_as_df_from_log_path(kfold_logs_path)
  alexnet_accuracy_df_infos[alexnet_procedure_path_infos_key] = accuracy_df.copy()
  print(alexnet_procedure_path_infos_key, accuracy_df.shape)




In [ ]:
fig, axes = plt.subplots(6, 6, figsize=(30, 30))

procedure_list = ['BLNC--ONL_ORG', \
                  'NON_BLNC--ONL_ORG', \
                  'BLNC--ORG_AUG_SEP',\
                  'NON_BLNC--ORG_AUG_SEP',\
                  'BLNC--ORG_AUG_MIX', \
                  'NON_BLNC--ORG_AUG_MIX']

axes_idx = 0

for row_idx in range(6):

  main_log_df = alexnet_accuracy_df_infos[f'{procedure_list[row_idx]}--ALL'].copy()

  sns.lineplot(data=main_log_df, x='Epoch', y='Ratio', hue="Phase", ax=axes[row_idx][0])

  procedure_components = procedure_list[row_idx].split('--')
  axes[row_idx][0].set_title(f'AlexNet {procedure_components[0]}, {procedure_components[1]}, ALL')
  axes_idx += 1

  main_log_df = alexnet_accuracy_df_infos[f'{procedure_list[row_idx]}--TD'].copy()

  sns.lineplot(data=main_log_df, x='Epoch', y='Ratio', hue="Phase", ax=axes[row_idx][1])

  procedure_components = procedure_list[row_idx].split('--')
  axes[row_idx][1].set_title(f'AlexNet {procedure_components[0]}, {procedure_components[1]}, TD')
  axes_idx += 1


  main_log_df = lstm_accuracy_df_infos[f'{procedure_list[row_idx]}--ALL'].copy()

  sns.lineplot(data=main_log_df, x='Epoch', y='Ratio', hue="Phase", ax=axes[row_idx][2])

  procedure_components = procedure_list[row_idx].split('--')
  axes[row_idx][2].set_title(f'LSTM {procedure_components[0]}, {procedure_components[1]}, ALL')
  axes_idx += 1

  main_log_df = lstm_accuracy_df_infos[f'{procedure_list[row_idx]}--TD'].copy()

  sns.lineplot(data=main_log_df, x='Epoch', y='Ratio', hue="Phase", ax=axes[row_idx][3])

  procedure_components = procedure_list[row_idx].split('--')
  axes[row_idx][3].set_title(f'LSTM {procedure_components[0]}, {procedure_components[1]}, TD')
  axes_idx += 1


  main_log_df = lstm_v2_accuracy_df_infos[f'{procedure_list[row_idx]}--ALL'].copy()

  sns.lineplot(data=main_log_df, x='Epoch', y='Ratio', hue="Phase", ax=axes[row_idx][4])

  procedure_components = procedure_list[row_idx].split('--')
  axes[row_idx][4].set_title(f'LSTM V2 {procedure_components[0]}, {procedure_components[1]}, ALL')
  axes_idx += 1

  main_log_df = lstm_v2_accuracy_df_infos[f'{procedure_list[row_idx]}--TD'].copy()

  sns.lineplot(data=main_log_df, x='Epoch', y='Ratio', hue="Phase", ax=axes[row_idx][5])

  procedure_components = procedure_list[row_idx].split('--')
  axes[row_idx][5].set_title(f'LSTM V2 {procedure_components[0]}, {procedure_components[1]}, TD')
  axes_idx += 1


# Adjust the spacing between plots
plt.tight_layout()

# Display the plot
plt.show()

# fig.savefig(f"{model_name}_{experiment_procedure_name}_{(i + 1)}KFold.png")

In [ ]:
fig, axes = plt.subplots(6, 2, figsize=(30, 30))

procedure_list = ['BLNC--ONL_ORG--ALL', \
                  'BLNC--ORG_AUG_SEP--ALL',\
                  'NON_BLNC--ORG_AUG_SEP--ALL',\
                  'BLNC--ORG_AUG_MIX--ALL', \
                  'NON_BLNC--ORG_AUG_MIX--ALL']

for row_idx in range(6):

  axes_idx = 0
  if row_idx < 5:

    main_log_df = lstm_v3_accuracy_df_infos[f'{procedure_list[row_idx]}--NON_RAW'].copy()

    sns.lineplot(data=main_log_df, x='Epoch', y='Ratio', hue="Phase", ax=axes[row_idx][axes_idx])

    procedure_components = procedure_list[row_idx].split('--')
    axes[row_idx][axes_idx].set_title(f'LSTM V3 {procedure_components[0]}, {procedure_components[1]}, {procedure_components[2]}')
    axes_idx += 1

    main_log_df = lstm_v4_accuracy_df_infos[procedure_list[row_idx]].copy()

    sns.lineplot(data=main_log_df, x='Epoch', y='Ratio', hue="Phase", ax=axes[row_idx][axes_idx])

    procedure_components = procedure_list[row_idx].split('--')
    axes[row_idx][axes_idx].set_title(f'LSTM V4 {procedure_components[0]}, {procedure_components[1]}, {procedure_components[2]}')
    axes_idx += 1

  else:

    main_log_df = lstm_v3_accuracy_df_infos[f'{procedure_list[3]}--RAW'].copy()

    sns.lineplot(data=main_log_df, x='Epoch', y='Ratio', hue="Phase", ax=axes[row_idx][axes_idx])

    procedure_components = procedure_list[3].split('--')
    axes[row_idx][axes_idx].set_title(f'LSTM V3 {procedure_components[0]}, {procedure_components[1]}, {procedure_components[2]}, RAW')
    axes_idx += 1


# Adjust the spacing between plots
plt.tight_layout()

# Display the plot
plt.show()

# fig.savefig(f"{model_name}_{experiment_procedure_name}_{(i + 1)}KFold.png")

In [ ]:
def get_train_loss_values_as_df_from_log_path(log_path):

  mean_loss_train = []
  mean_loss_val = []

  kfold_range = int((len(os.listdir(log_path)) - 1) / 5)

  for i in range(kfold_range):

    log_dir = f'{log_path}/Loss - {(i + 1)}_val/loss'
    ratio_values = tflog2pandas(os.path.join(log_dir, os.listdir(log_dir)[0]), f'Loss - {(i + 1)}')
    if i > 0:
      if len(mean_loss_val[0]) == len(ratio_values):
        mean_loss_val.append(list(ratio_values))
    else:
      mean_loss_val.append(list(ratio_values))

    log_dir = f'{log_path}/Loss - {(i + 1)}_train/loss'
    ratio_values = tflog2pandas(os.path.join(log_dir, os.listdir(log_dir)[0]), f'Loss - {(i + 1)}')
    if i > 0:
      if len(mean_loss_val[0]) == len(ratio_values):
        mean_loss_train.append(list(ratio_values))
    else:
      mean_loss_train.append(list(ratio_values))


  main_log_df = pd.DataFrame()

  log_df = pd.DataFrame()
  log_df['Ratio'] = np.mean(mean_loss_val, axis=0)
  log_df['Phase'] = ['Validation' for ratio_idx in range(len(log_df['Ratio']))]
  log_df['Epoch'] = [epoch_idx + 1 for epoch_idx in range(len(log_df['Ratio']))]
  main_log_df = pd.concat([main_log_df, log_df])

  log_df = pd.DataFrame()
  log_df['Ratio'] = np.mean(mean_loss_train, axis=0)
  log_df['Phase'] = ['Train' for ratio_idx in range(len(log_df['Ratio']))]
  log_df['Epoch'] = [epoch_idx + 1 for epoch_idx in range(len(log_df['Ratio']))]
  main_log_df = pd.concat([main_log_df, log_df])

  return main_log_df


In [ ]:

lstm_loss_df_infos = {}

for lstm_procedure_path_infos_key in lstm_procedure_path_infos.keys():
  kfold_logs_path = f'{EXPR_PATH}/{lstm_procedure_path_infos[lstm_procedure_path_infos_key]}/'

  sample_files = [sample_file.name for sample_file in sorted(Path(kfold_logs_path).iterdir(), key=os.path.getmtime) if sample_file.is_dir() and sample_file.name.startswith('run')]

  kfold_logs_path = os.path.join(kfold_logs_path, sample_files[-1])
  for __ in range(4):
    kfold_logs_path = os.path.join(kfold_logs_path, os.listdir(kfold_logs_path)[0])

  loss_df = get_train_loss_values_as_df_from_log_path(kfold_logs_path)
  lstm_loss_df_infos[lstm_procedure_path_infos_key] = loss_df.copy()
  print(lstm_procedure_path_infos_key, loss_df.shape)




lstm_v2_loss_df_infos = {}

for lstm_v2_procedure_path_infos_key in lstm_v2_procedure_path_infos.keys():
  kfold_logs_path = f'{EXPR_PATH}/{lstm_v2_procedure_path_infos[lstm_v2_procedure_path_infos_key]}/'

  sample_files = [sample_file.name for sample_file in sorted(Path(kfold_logs_path).iterdir(), key=os.path.getmtime) if sample_file.is_dir() and sample_file.name.startswith('run')]

  kfold_logs_path = os.path.join(kfold_logs_path, sample_files[-1])
  for __ in range(4):
    kfold_logs_path = os.path.join(kfold_logs_path, os.listdir(kfold_logs_path)[0])

  loss_df = get_train_loss_values_as_df_from_log_path(kfold_logs_path)
  lstm_v2_loss_df_infos[lstm_v2_procedure_path_infos_key] = loss_df.copy()
  print(lstm_v2_procedure_path_infos_key, loss_df.shape)




alexnet_loss_df_infos = {}

for alexnet_procedure_path_infos_key in alexnet_procedure_path_infos.keys():
  kfold_logs_path = f'{EXPR_PATH}/{alexnet_procedure_path_infos[alexnet_procedure_path_infos_key]}/'

  sample_files = [sample_file.name for sample_file in sorted(Path(kfold_logs_path).iterdir(), key=os.path.getmtime) if sample_file.is_dir() and sample_file.name.startswith('run')]

  kfold_logs_path = os.path.join(kfold_logs_path, sample_files[-1])
  for __ in range(4):
    kfold_logs_path = os.path.join(kfold_logs_path, os.listdir(kfold_logs_path)[0])

  loss_df = get_train_loss_values_as_df_from_log_path(kfold_logs_path)
  alexnet_loss_df_infos[alexnet_procedure_path_infos_key] = loss_df.copy()
  print(alexnet_procedure_path_infos_key, loss_df.shape)




In [ ]:
fig, axes = plt.subplots(6, 6, figsize=(30, 30))

procedure_list = ['BLNC--ONL_ORG', \
                  'NON_BLNC--ONL_ORG', \
                  'BLNC--ORG_AUG_SEP',\
                  'NON_BLNC--ORG_AUG_SEP',\
                  'BLNC--ORG_AUG_MIX', \
                  'NON_BLNC--ORG_AUG_MIX']

axes_idx = 0

for row_idx in range(6):

  main_log_df = alexnet_loss_df_infos[f'{procedure_list[row_idx]}--ALL'].copy()

  sns.lineplot(data=main_log_df, x='Epoch', y='Ratio', hue="Phase", ax=axes[row_idx][0])

  procedure_components = procedure_list[row_idx].split('--')
  axes[row_idx][0].set_title(f'AlexNet {procedure_components[0]}, {procedure_components[1]}, ALL')
  axes_idx += 1

  main_log_df = alexnet_loss_df_infos[f'{procedure_list[row_idx]}--TD'].copy()

  sns.lineplot(data=main_log_df, x='Epoch', y='Ratio', hue="Phase", ax=axes[row_idx][1])

  procedure_components = procedure_list[row_idx].split('--')
  axes[row_idx][1].set_title(f'AlexNet {procedure_components[0]}, {procedure_components[1]}, TD')
  axes_idx += 1


  main_log_df = lstm_loss_df_infos[f'{procedure_list[row_idx]}--ALL'].copy()

  sns.lineplot(data=main_log_df, x='Epoch', y='Ratio', hue="Phase", ax=axes[row_idx][2])

  procedure_components = procedure_list[row_idx].split('--')
  axes[row_idx][2].set_title(f'LSTM {procedure_components[0]}, {procedure_components[1]}, ALL')
  axes_idx += 1

  main_log_df = lstm_loss_df_infos[f'{procedure_list[row_idx]}--TD'].copy()

  sns.lineplot(data=main_log_df, x='Epoch', y='Ratio', hue="Phase", ax=axes[row_idx][3])

  procedure_components = procedure_list[row_idx].split('--')
  axes[row_idx][3].set_title(f'LSTM {procedure_components[0]}, {procedure_components[1]}, TD')
  axes_idx += 1


  main_log_df = lstm_v2_loss_df_infos[f'{procedure_list[row_idx]}--ALL'].copy()

  sns.lineplot(data=main_log_df, x='Epoch', y='Ratio', hue="Phase", ax=axes[row_idx][4])

  procedure_components = procedure_list[row_idx].split('--')
  axes[row_idx][4].set_title(f'LSTM V2 {procedure_components[0]}, {procedure_components[1]}, ALL')
  axes_idx += 1

  main_log_df = lstm_v2_loss_df_infos[f'{procedure_list[row_idx]}--TD'].copy()

  sns.lineplot(data=main_log_df, x='Epoch', y='Ratio', hue="Phase", ax=axes[row_idx][5])

  procedure_components = procedure_list[row_idx].split('--')
  axes[row_idx][5].set_title(f'LSTM V2 {procedure_components[0]}, {procedure_components[1]}, TD')
  axes_idx += 1


# Adjust the spacing between plots
plt.tight_layout()

# Display the plot
plt.show()

# fig.savefig(f"{model_name}_{experiment_procedure_name}_{(i + 1)}KFold.png")

In [ ]:

lstm_v3_loss_df_infos = {}

for lstm_v3_procedure_path_infos_key in lstm_v3_procedure_path_infos.keys():
  kfold_logs_path = f'{EXPR_PATH}/{lstm_v3_procedure_path_infos[lstm_v3_procedure_path_infos_key]}/'

  sample_files = [sample_file.name for sample_file in sorted(Path(kfold_logs_path).iterdir(), key=os.path.getmtime) if sample_file.is_dir() and sample_file.name.startswith('run')]

  kfold_logs_path = os.path.join(kfold_logs_path, sample_files[-1])
  for __ in range(4):
    kfold_logs_path = os.path.join(kfold_logs_path, os.listdir(kfold_logs_path)[0])

  loss_df = get_train_loss_values_as_df_from_log_path(kfold_logs_path)
  lstm_v3_loss_df_infos[lstm_v3_procedure_path_infos_key] = loss_df.copy()
  print(lstm_v3_procedure_path_infos_key, loss_df.shape)




lstm_v4_loss_df_infos = {}

for lstm_v4_procedure_path_infos_key in lstm_v4_procedure_path_infos.keys():
  kfold_logs_path = f'{EXPR_PATH}/{lstm_v4_procedure_path_infos[lstm_v4_procedure_path_infos_key]}/'

  sample_files = [sample_file.name for sample_file in sorted(Path(kfold_logs_path).iterdir(), key=os.path.getmtime) if sample_file.is_dir() and sample_file.name.startswith('run')]

  kfold_logs_path = os.path.join(kfold_logs_path, sample_files[-1])
  for __ in range(4):
    kfold_logs_path = os.path.join(kfold_logs_path, os.listdir(kfold_logs_path)[0])

  loss_df = get_train_loss_values_as_df_from_log_path(kfold_logs_path)
  lstm_v4_loss_df_infos[lstm_v4_procedure_path_infos_key] = loss_df.copy()
  print(lstm_v4_procedure_path_infos_key, loss_df.shape)




In [ ]:
fig, axes = plt.subplots(6, 2, figsize=(30, 30))

procedure_list = ['BLNC--ONL_ORG--ALL', \
                  'BLNC--ORG_AUG_SEP--ALL',\
                  'NON_BLNC--ORG_AUG_SEP--ALL',\
                  'BLNC--ORG_AUG_MIX--ALL', \
                  'NON_BLNC--ORG_AUG_MIX--ALL']

for row_idx in range(6):

  axes_idx = 0
  if row_idx < 5:

    main_log_df = lstm_v3_loss_df_infos[f'{procedure_list[row_idx]}--NON_RAW'].copy()

    sns.lineplot(data=main_log_df, x='Epoch', y='Ratio', hue="Phase", ax=axes[row_idx][axes_idx])

    procedure_components = procedure_list[row_idx].split('--')
    axes[row_idx][axes_idx].set_title(f'LSTM V3 {procedure_components[0]}, {procedure_components[1]}, {procedure_components[2]}')
    axes_idx += 1

    main_log_df = lstm_v4_loss_df_infos[procedure_list[row_idx]].copy()

    sns.lineplot(data=main_log_df, x='Epoch', y='Ratio', hue="Phase", ax=axes[row_idx][axes_idx])

    procedure_components = procedure_list[row_idx].split('--')
    axes[row_idx][axes_idx].set_title(f'LSTM V4 {procedure_components[0]}, {procedure_components[1]}, {procedure_components[2]}')
    axes_idx += 1

  else:

    main_log_df = lstm_v3_loss_df_infos[f'{procedure_list[3]}--RAW'].copy()

    sns.lineplot(data=main_log_df, x='Epoch', y='Ratio', hue="Phase", ax=axes[row_idx][axes_idx])

    procedure_components = procedure_list[3].split('--')
    axes[row_idx][axes_idx].set_title(f'LSTM V3 {procedure_components[0]}, {procedure_components[1]}, {procedure_components[2]}, RAW')
    axes_idx += 1


# Adjust the spacing between plots
plt.tight_layout()

# Display the plot
plt.show()

# fig.savefig(f"{model_name}_{experiment_procedure_name}_{(i + 1)}KFold.png")

In [ ]:
def get_train_other_values_as_df_from_log_path(log_path):

  kfold_range = int((len(os.listdir(log_path)) - 1) / 5)
  main_log_df = pd.DataFrame()

  mean_f1score_val = []
  for i in range(kfold_range):

    log_dir = f'{log_path}/Others - {(i + 1)}_val/f1score'
    ratio_values = tflog2pandas(os.path.join(log_dir, os.listdir(log_dir)[0]), f'Others - {(i + 1)}')
    if i > 0:
      if len(mean_f1score_val[0]) == len(ratio_values):
        mean_f1score_val.append(list(ratio_values))
    else:
      mean_f1score_val.append(list(ratio_values))

  log_df = pd.DataFrame()
  log_df['Ratio'] = np.mean(mean_f1score_val, axis=0)
  log_df['Phase'] = ['F1 Score' for ratio_idx in range(len(log_df['Ratio']))]
  log_df['Epoch'] = [epoch_idx + 1 for epoch_idx in range(len(log_df['Ratio']))]
  main_log_df = pd.concat([main_log_df, log_df])


  mean_precision_val = []
  for i in range(kfold_range):

    log_dir = f'{log_path}/Others - {(i + 1)}_val/precision'
    ratio_values = tflog2pandas(os.path.join(log_dir, os.listdir(log_dir)[0]), f'Others - {(i + 1)}')
    if i > 0:
      if len(mean_precision_val[0]) == len(ratio_values):
        mean_precision_val.append(list(ratio_values))
    else:
      mean_precision_val.append(list(ratio_values))

  log_df = pd.DataFrame()
  log_df['Ratio'] = np.mean(mean_precision_val, axis=0)
  log_df['Phase'] = ['Precision' for ratio_idx in range(len(log_df['Ratio']))]
  log_df['Epoch'] = [epoch_idx + 1 for epoch_idx in range(len(log_df['Ratio']))]
  main_log_df = pd.concat([main_log_df, log_df])


  mean_recall_val = []
  for i in range(kfold_range):

    log_dir = f'{log_path}/Others - {(i + 1)}_val/recall'
    ratio_values = tflog2pandas(os.path.join(log_dir, os.listdir(log_dir)[0]), f'Others - {(i + 1)}')
    if i > 0:
      if len(mean_f1score_val[0]) == len(ratio_values):
        mean_f1score_val.append(list(ratio_values))
    else:
      mean_f1score_val.append(list(ratio_values))

  log_df = pd.DataFrame()
  log_df['Ratio'] = np.mean(mean_f1score_val, axis=0)
  log_df['Phase'] = ['Recall' for ratio_idx in range(len(log_df['Ratio']))]
  log_df['Epoch'] = [epoch_idx + 1 for epoch_idx in range(len(log_df['Ratio']))]
  main_log_df = pd.concat([main_log_df, log_df])


  return main_log_df


In [ ]:

lstm_other_df_infos = {}

for lstm_procedure_path_infos_key in lstm_procedure_path_infos.keys():
  kfold_logs_path = f'{EXPR_PATH}/{lstm_procedure_path_infos[lstm_procedure_path_infos_key]}/'

  sample_files = [sample_file.name for sample_file in sorted(Path(kfold_logs_path).iterdir(), key=os.path.getmtime) if sample_file.is_dir() and sample_file.name.startswith('run')]

  kfold_logs_path = os.path.join(kfold_logs_path, sample_files[-1])
  for __ in range(4):
    kfold_logs_path = os.path.join(kfold_logs_path, os.listdir(kfold_logs_path)[0])

  other_df = get_train_other_values_as_df_from_log_path(kfold_logs_path)
  lstm_other_df_infos[lstm_procedure_path_infos_key] = other_df.copy()
  print(lstm_procedure_path_infos_key, other_df.shape)




lstm_v2_other_df_infos = {}

for lstm_v2_procedure_path_infos_key in lstm_v2_procedure_path_infos.keys():
  kfold_logs_path = f'{EXPR_PATH}/{lstm_v2_procedure_path_infos[lstm_v2_procedure_path_infos_key]}/'

  sample_files = [sample_file.name for sample_file in sorted(Path(kfold_logs_path).iterdir(), key=os.path.getmtime) if sample_file.is_dir() and sample_file.name.startswith('run')]

  kfold_logs_path = os.path.join(kfold_logs_path, sample_files[-1])
  for __ in range(4):
    kfold_logs_path = os.path.join(kfold_logs_path, os.listdir(kfold_logs_path)[0])

  other_df = get_train_other_values_as_df_from_log_path(kfold_logs_path)
  lstm_v2_other_df_infos[lstm_v2_procedure_path_infos_key] = other_df.copy()
  print(lstm_v2_procedure_path_infos_key, other_df.shape)




alexnet_other_df_infos = {}

for alexnet_procedure_path_infos_key in alexnet_procedure_path_infos.keys():
  kfold_logs_path = f'{EXPR_PATH}/{alexnet_procedure_path_infos[alexnet_procedure_path_infos_key]}/'

  sample_files = [sample_file.name for sample_file in sorted(Path(kfold_logs_path).iterdir(), key=os.path.getmtime) if sample_file.is_dir() and sample_file.name.startswith('run')]

  kfold_logs_path = os.path.join(kfold_logs_path, sample_files[-1])
  for __ in range(4):
    kfold_logs_path = os.path.join(kfold_logs_path, os.listdir(kfold_logs_path)[0])

  other_df = get_train_other_values_as_df_from_log_path(kfold_logs_path)
  alexnet_other_df_infos[alexnet_procedure_path_infos_key] = other_df.copy()
  print(alexnet_procedure_path_infos_key, other_df.shape)




In [ ]:
fig, axes = plt.subplots(6, 6, figsize=(30, 30))

procedure_list = ['BLNC--ONL_ORG', \
                  'NON_BLNC--ONL_ORG', \
                  'BLNC--ORG_AUG_SEP',\
                  'NON_BLNC--ORG_AUG_SEP',\
                  'BLNC--ORG_AUG_MIX', \
                  'NON_BLNC--ORG_AUG_MIX']

axes_idx = 0

for row_idx in range(6):

  main_log_df = alexnet_other_df_infos[f'{procedure_list[row_idx]}--ALL'].copy()

  sns.lineplot(data=main_log_df, x='Epoch', y='Ratio', hue="Phase", ax=axes[row_idx][0])

  procedure_components = procedure_list[row_idx].split('--')
  axes[row_idx][0].set_title(f'AlexNet {procedure_components[0]}, {procedure_components[1]}, ALL')
  axes_idx += 1

  main_log_df = alexnet_other_df_infos[f'{procedure_list[row_idx]}--TD'].copy()

  sns.lineplot(data=main_log_df, x='Epoch', y='Ratio', hue="Phase", ax=axes[row_idx][1])

  procedure_components = procedure_list[row_idx].split('--')
  axes[row_idx][1].set_title(f'AlexNet {procedure_components[0]}, {procedure_components[1]}, TD')
  axes_idx += 1


  main_log_df = lstm_other_df_infos[f'{procedure_list[row_idx]}--ALL'].copy()

  sns.lineplot(data=main_log_df, x='Epoch', y='Ratio', hue="Phase", ax=axes[row_idx][2])

  procedure_components = procedure_list[row_idx].split('--')
  axes[row_idx][2].set_title(f'LSTM {procedure_components[0]}, {procedure_components[1]}, ALL')
  axes_idx += 1

  main_log_df = lstm_other_df_infos[f'{procedure_list[row_idx]}--TD'].copy()

  sns.lineplot(data=main_log_df, x='Epoch', y='Ratio', hue="Phase", ax=axes[row_idx][3])

  procedure_components = procedure_list[row_idx].split('--')
  axes[row_idx][3].set_title(f'LSTM {procedure_components[0]}, {procedure_components[1]}, TD')
  axes_idx += 1


  main_log_df = lstm_v2_other_df_infos[f'{procedure_list[row_idx]}--ALL'].copy()

  sns.lineplot(data=main_log_df, x='Epoch', y='Ratio', hue="Phase", ax=axes[row_idx][4])

  procedure_components = procedure_list[row_idx].split('--')
  axes[row_idx][4].set_title(f'LSTM V2 {procedure_components[0]}, {procedure_components[1]}, ALL')
  axes_idx += 1

  main_log_df = lstm_v2_other_df_infos[f'{procedure_list[row_idx]}--TD'].copy()

  sns.lineplot(data=main_log_df, x='Epoch', y='Ratio', hue="Phase", ax=axes[row_idx][5])

  procedure_components = procedure_list[row_idx].split('--')
  axes[row_idx][5].set_title(f'LSTM V2 {procedure_components[0]}, {procedure_components[1]}, TD')
  axes_idx += 1


# Adjust the spacing between plots
plt.tight_layout()

# Display the plot
plt.show()

# fig.savefig(f"{model_name}_{experiment_procedure_name}_{(i + 1)}KFold.png")

In [ ]:

lstm_v3_other_df_infos = {}

for lstm_v3_procedure_path_infos_key in lstm_v3_procedure_path_infos.keys():
  kfold_logs_path = f'{EXPR_PATH}/{lstm_v3_procedure_path_infos[lstm_v3_procedure_path_infos_key]}/'

  sample_files = [sample_file.name for sample_file in sorted(Path(kfold_logs_path).iterdir(), key=os.path.getmtime) if sample_file.is_dir() and sample_file.name.startswith('run')]

  kfold_logs_path = os.path.join(kfold_logs_path, sample_files[-1])
  for __ in range(4):
    kfold_logs_path = os.path.join(kfold_logs_path, os.listdir(kfold_logs_path)[0])

  other_df = get_train_other_values_as_df_from_log_path(kfold_logs_path)
  lstm_v3_other_df_infos[lstm_v3_procedure_path_infos_key] = other_df.copy()
  print(lstm_v3_procedure_path_infos_key, other_df.shape)




lstm_v4_other_df_infos = {}

for lstm_v4_procedure_path_infos_key in lstm_v4_procedure_path_infos.keys():
  kfold_logs_path = f'{EXPR_PATH}/{lstm_v4_procedure_path_infos[lstm_v4_procedure_path_infos_key]}/'

  sample_files = [sample_file.name for sample_file in sorted(Path(kfold_logs_path).iterdir(), key=os.path.getmtime) if sample_file.is_dir() and sample_file.name.startswith('run')]

  kfold_logs_path = os.path.join(kfold_logs_path, sample_files[-1])
  for __ in range(4):
    kfold_logs_path = os.path.join(kfold_logs_path, os.listdir(kfold_logs_path)[0])

  other_df = get_train_other_values_as_df_from_log_path(kfold_logs_path)
  lstm_v4_other_df_infos[lstm_v4_procedure_path_infos_key] = other_df.copy()
  print(lstm_v4_procedure_path_infos_key, other_df.shape)




In [ ]:
fig, axes = plt.subplots(6, 2, figsize=(30, 30))

procedure_list = ['BLNC--ONL_ORG--ALL', \
                  'BLNC--ORG_AUG_SEP--ALL',\
                  'NON_BLNC--ORG_AUG_SEP--ALL',\
                  'BLNC--ORG_AUG_MIX--ALL', \
                  'NON_BLNC--ORG_AUG_MIX--ALL']

for row_idx in range(6):

  axes_idx = 0
  if row_idx < 5:

    main_log_df = lstm_v3_other_df_infos[f'{procedure_list[row_idx]}--NON_RAW'].copy()

    sns.lineplot(data=main_log_df, x='Epoch', y='Ratio', hue="Phase", ax=axes[row_idx][axes_idx])

    procedure_components = procedure_list[row_idx].split('--')
    axes[row_idx][axes_idx].set_title(f'LSTM V3 {procedure_components[0]}, {procedure_components[1]}, {procedure_components[2]}')
    axes_idx += 1

    main_log_df = lstm_v4_other_df_infos[procedure_list[row_idx]].copy()

    sns.lineplot(data=main_log_df, x='Epoch', y='Ratio', hue="Phase", ax=axes[row_idx][axes_idx])

    procedure_components = procedure_list[row_idx].split('--')
    axes[row_idx][axes_idx].set_title(f'LSTM V4 {procedure_components[0]}, {procedure_components[1]}, {procedure_components[2]}')
    axes_idx += 1

  else:

    main_log_df = lstm_v3_other_df_infos[f'{procedure_list[3]}--RAW'].copy()

    sns.lineplot(data=main_log_df, x='Epoch', y='Ratio', hue="Phase", ax=axes[row_idx][axes_idx])

    procedure_components = procedure_list[3].split('--')
    axes[row_idx][axes_idx].set_title(f'LSTM V3 {procedure_components[0]}, {procedure_components[1]}, {procedure_components[2]}, RAW')
    axes_idx += 1


# Adjust the spacing between plots
plt.tight_layout()

# Display the plot
plt.show()

# fig.savefig(f"{model_name}_{experiment_procedure_name}_{(i + 1)}KFold.png")

In [ ]:

lstm_test_dataset_df_infos = {}
print('LSTM Test Dataset Collectting')
for lstm_procedure_path_infos_key in lstm_procedure_path_infos.keys():
  kfold_logs_path = f'{EXPR_PATH}/{lstm_procedure_path_infos[lstm_procedure_path_infos_key]}/'

  INPUT_DIR = os.path.join(kfold_logs_path, sorted([exp_fold for exp_fold in os.listdir(kfold_logs_path) if exp_fold.startswith('test_dataset')])[-1])

  df = pd.read_csv(INPUT_DIR)
  df = df.drop('Unnamed: 0', axis=1)
  df['Diagnose'] = df['Diagnose'].apply(lambda x: 'No Stress' if x == 0 else 'Stress')
  print(lstm_procedure_path_infos_key, df.columns)
  lstm_test_dataset_df_infos[lstm_procedure_path_infos_key] = df.copy()



lstm_v2_test_dataset_df_infos = {}
print('LSTM Test Dataset Collectting')
for lstm_v2_procedure_path_infos_key in lstm_v2_procedure_path_infos.keys():
  kfold_logs_path = f'{EXPR_PATH}/{lstm_v2_procedure_path_infos[lstm_v2_procedure_path_infos_key]}/'

  INPUT_DIR = os.path.join(kfold_logs_path, sorted([exp_fold for exp_fold in os.listdir(kfold_logs_path) if exp_fold.startswith('test_dataset')])[-1])

  df = pd.read_csv(INPUT_DIR)
  df = df.drop('Unnamed: 0', axis=1)
  df['Diagnose'] = df['Diagnose'].apply(lambda x: 'No Stress' if x == 0 else 'Stress')
  print(lstm_v2_procedure_path_infos_key, df.columns)
  lstm_v2_test_dataset_df_infos[lstm_v2_procedure_path_infos_key] = df.copy()




alexnet_test_dataset_df_infos = {}
print('AlexNet Test Dataset Collectting')
for alexnet_procedure_path_infos_key in alexnet_procedure_path_infos.keys():
  kfold_logs_path = f'{EXPR_PATH}/{alexnet_procedure_path_infos[alexnet_procedure_path_infos_key]}/'

  INPUT_DIR = os.path.join(kfold_logs_path, sorted([exp_fold for exp_fold in os.listdir(kfold_logs_path) if exp_fold.startswith('test_dataset')])[-1])

  df = pd.read_csv(INPUT_DIR)
  df = df.drop('Unnamed: 0', axis=1)
  df['Diagnose'] = df['Diagnose'].apply(lambda x: 'No Stress' if x == 0 else 'Stress')
  print(alexnet_procedure_path_infos_key, df.columns)
  alexnet_test_dataset_df_infos[alexnet_procedure_path_infos_key] = df.copy()




In [ ]:

lstm_v3_test_dataset_df_infos = {}
print('LSTM V3 Test Dataset Collectting')
for lstm_v3_procedure_path_infos_key in lstm_v3_procedure_path_infos.keys():
  kfold_logs_path = f'{EXPR_PATH}/{lstm_v3_procedure_path_infos[lstm_v3_procedure_path_infos_key]}/'

  INPUT_DIR = os.path.join(kfold_logs_path, sorted([exp_fold for exp_fold in os.listdir(kfold_logs_path) if exp_fold.startswith('test_dataset')])[-1])

  df = pd.read_csv(INPUT_DIR)
  df = df.drop('Unnamed: 0', axis=1)
  df['Diagnose'] = df['Diagnose'].apply(lambda x: 'No Stress' if x == 0 else 'Stress')
  print(lstm_v3_procedure_path_infos_key, df.columns)
  lstm_v3_test_dataset_df_infos[lstm_v3_procedure_path_infos_key] = df.copy()

lstm_v4_test_dataset_df_infos = {}
print('LSTM V4 Test Dataset Collectting')
for lstm_v4_procedure_path_infos_key in lstm_v4_procedure_path_infos.keys():
  kfold_logs_path = f'{EXPR_PATH}/{lstm_v4_procedure_path_infos[lstm_v4_procedure_path_infos_key]}/'

  INPUT_DIR = os.path.join(kfold_logs_path, sorted([exp_fold for exp_fold in os.listdir(kfold_logs_path) if exp_fold.startswith('test_dataset')])[-1])

  df = pd.read_csv(INPUT_DIR)
  df = df.drop('Unnamed: 0', axis=1)
  df['Diagnose'] = df['Diagnose'].apply(lambda x: 'No Stress' if x == 0 else 'Stress')
  print(lstm_v4_procedure_path_infos_key, df.columns)
  lstm_v4_test_dataset_df_infos[lstm_v4_procedure_path_infos_key] = df.copy()



In [ ]:
# from this site https://christianbernecker.medium.com/how-to-create-a-confusion-matrix-in-pytorch-38d06a7f04b7
def test_model_without_save_as_image(model , model_name,device, test_loader, result_classes):

        y_pred = []
        y_true = []
        correct = 0
        total = 0
        with torch.no_grad():
            for data in test_loader:

                images, labels = data[0].to(device), data[1].to(device)
                outputs = model(images)

                # _, predicted = torch.max(outputs.data, 1)
                _, predicted = torch.max(outputs, 1)
                output = (predicted).data.cpu().numpy()
                y_pred.extend(output) # Save Prediction

                # Convert labels to class indices before comparison:
                labels_indices = torch.argmax(labels, dim=1)  # Get indices from one-hot labels
                labels_np = labels_indices.data.cpu().numpy()  # Convert to NumPy for y_true
                y_true.extend(labels_np) # Save Truth

                total += labels_np.shape[0] # Get the number of labels using .shape[0] for NumPy array
                # total += labels.size(0)
                # Compare predicted indices with true indices
                correct += (predicted == labels_indices).sum().item()

        ty_pred = torch.tensor(y_pred)
        ty_true = torch.tensor(y_true)

        # Build confusion matrix
        cf_matrix = confusion_matrix(y_true, y_pred)
        df_cm = pd.DataFrame(cf_matrix, index = [i for i in result_classes],
                            columns = [i for i in result_classes])

        return (df_cm, correct, total, binary_accuracy(ty_pred, ty_true), binary_recall(ty_pred, ty_true), binary_precision(ty_pred, ty_true), binary_f1_score(ty_pred, ty_true))


In [ ]:

lstm_best_model_infos = {}
print('LSTM Test Dataset Collectting')
for lstm_procedure_path_infos_key in lstm_procedure_path_infos.keys():

  model_expr_path = os.path.join(EXPR_PATH, lstm_procedure_path_infos[lstm_procedure_path_infos_key])
  latest_exp_folder = os.path.join(model_expr_path, sorted([exp_fold for exp_fold in os.listdir(model_expr_path) if exp_fold.startswith('modelPerformance')])[-1])

  for __ in range(2):
    latest_exp_folder = os.path.join(latest_exp_folder, os.listdir(latest_exp_folder)[0])

  best_model=torch.load(os.path.join(latest_exp_folder, (sorted(os.listdir(latest_exp_folder))[-1])))
  lstm_best_model_infos[lstm_procedure_path_infos_key] = copy.deepcopy(best_model)



lstm_v2_best_model_infos = {}
print('LSTM V2 Test Dataset Collectting')
for lstm_v2_procedure_path_infos_key in lstm_v2_procedure_path_infos.keys():

  model_expr_path = os.path.join(EXPR_PATH, lstm_v2_procedure_path_infos[lstm_v2_procedure_path_infos_key])
  latest_exp_folder = os.path.join(model_expr_path, sorted([exp_fold for exp_fold in os.listdir(model_expr_path) if exp_fold.startswith('modelPerformance')])[-1])

  for __ in range(2):
    latest_exp_folder = os.path.join(latest_exp_folder, os.listdir(latest_exp_folder)[0])

  best_model=torch.load(os.path.join(latest_exp_folder, (sorted(os.listdir(latest_exp_folder))[-1])))
  lstm_v2_best_model_infos[lstm_v2_procedure_path_infos_key] = copy.deepcopy(best_model)


alexnet_best_model_infos = {}
print('AlexNet Test Dataset Collectting')
for alexnet_test_dataset_df_infos_keys in alexnet_test_dataset_df_infos.keys():
  # df = alexnet_test_dataset_df_infos[alexnet_test_dataset_df_infos_keys].copy()
  # df_without_diagnose = df.drop('Diagnose', axis=1)
  # dataset_test = get_samples_as_dataloader(df_without_diagnose.values, df['Diagnose'].values, True)

  model_expr_path = os.path.join(EXPR_PATH, alexnet_procedure_path_infos[alexnet_test_dataset_df_infos_keys])
  latest_exp_folder = os.path.join(model_expr_path, sorted([exp_fold for exp_fold in os.listdir(model_expr_path) if exp_fold.startswith('modelPerformance')])[-1])

  for __ in range(2):
    latest_exp_folder = os.path.join(latest_exp_folder, os.listdir(latest_exp_folder)[0])

  best_model=torch.load(os.path.join(latest_exp_folder, (sorted(os.listdir(latest_exp_folder))[-1])))
  alexnet_best_model_infos[alexnet_test_dataset_df_infos_keys] = copy.deepcopy(best_model)


In [ ]:

lstm_v3_best_model_infos = {}
print('LSTM V3 Model Collectting')
for lstm_v3_procedure_path_infos_key in lstm_v3_procedure_path_infos.keys():

  model_expr_path = os.path.join(EXPR_PATH, lstm_v3_procedure_path_infos[lstm_v3_procedure_path_infos_key])
  latest_exp_folder = os.path.join(model_expr_path, sorted([exp_fold for exp_fold in os.listdir(model_expr_path) if exp_fold.startswith('modelPerformance')])[-1])

  for __ in range(2):
    latest_exp_folder = os.path.join(latest_exp_folder, os.listdir(latest_exp_folder)[0])

  best_model=torch.load(os.path.join(latest_exp_folder, (sorted(os.listdir(latest_exp_folder))[-1])))
  lstm_v3_best_model_infos[lstm_v3_procedure_path_infos_key] = copy.deepcopy(best_model)


lstm_v4_best_model_infos = {}
print('LSTM V4 Model Collectting')
for lstm_v4_procedure_path_infos_key in lstm_v4_procedure_path_infos.keys():

  model_expr_path = os.path.join(EXPR_PATH, lstm_v4_procedure_path_infos[lstm_v4_procedure_path_infos_key])
  latest_exp_folder = os.path.join(model_expr_path, sorted([exp_fold for exp_fold in os.listdir(model_expr_path) if exp_fold.startswith('modelPerformance')])[-1])

  for __ in range(2):
    latest_exp_folder = os.path.join(latest_exp_folder, os.listdir(latest_exp_folder)[0])

  best_model=torch.load(os.path.join(latest_exp_folder, (sorted(os.listdir(latest_exp_folder))[-1])))
  lstm_v4_best_model_infos[lstm_v4_procedure_path_infos_key] = copy.deepcopy(best_model)


In [ ]:

lstm_test_results_infos = {}
print('LSTM Test Dataset Collectting')
for lstm_procedure_path_infos_key in lstm_procedure_path_infos.keys():

  df = lstm_test_dataset_df_infos[lstm_procedure_path_infos_key].copy()
  df_without_diagnose = df.drop('Diagnose', axis=1)
  y_test = df['Diagnose'].values
  dataset_test = get_samples_as_dataloader(df_without_diagnose.values, y_test, True)

  classes = np.unique(y_test)
  le = OneHotEncoder()
  classes_idx = le.fit_transform(classes.reshape(-1, 1)).toarray()

  class_dict = { cls : cls_idx for cls, cls_idx in zip(classes, classes_idx)}
  # sorted_class_list = sorted(class_dict.items(), key=lambda kv: (kv[1], kv[0]))
  # sorted_class_labels = [d_key for d_key in dict(sorted_class_list).keys()]
  sorted_class_labels = class_dict

  dataset_test_loader = DataLoader(dataset_test, shuffle=True, batch_size=10 if 'MIX' in lstm_procedure_path_infos_key else 5)
  lstm_test_results_infos[lstm_procedure_path_infos_key] = test_model_without_save_as_image(lstm_best_model_infos[lstm_procedure_path_infos_key], f'LSTM_{lstm_procedure_path_infos_key}','cpu', dataset_test_loader, sorted_class_labels)



lstm_v2_test_results_infos = {}
print('LSTM V2 Test Dataset Collectting')
for lstm_v2_procedure_path_infos_key in lstm_v2_procedure_path_infos.keys():

  df = lstm_v2_test_dataset_df_infos[lstm_v2_procedure_path_infos_key].copy()
  df_without_diagnose = df.drop('Diagnose', axis=1)
  y_test = df['Diagnose'].values
  dataset_test = get_samples_as_dataloader(df_without_diagnose.values, y_test, True)

  classes = np.unique(y_test)
  le = OneHotEncoder()
  classes_idx = le.fit_transform(classes.reshape(-1, 1)).toarray()

  class_dict = { cls : cls_idx for cls, cls_idx in zip(classes, classes_idx)}
  # sorted_class_list = sorted(class_dict.items(), key=lambda kv: (kv[1], kv[0]))
  # sorted_class_labels = [d_key for d_key in dict(sorted_class_list).keys()]
  sorted_class_labels = class_dict

  dataset_test_loader = DataLoader(dataset_test, shuffle=True, batch_size=10 if 'MIX' in lstm_v2_procedure_path_infos_key else 5)
  lstm_v2_test_results_infos[lstm_v2_procedure_path_infos_key] = test_model_without_save_as_image(lstm_v2_best_model_infos[lstm_v2_procedure_path_infos_key], f'LSTM_V2_{lstm_v2_procedure_path_infos_key}','cpu', dataset_test_loader, sorted_class_labels)


alexnet_test_results_infos = {}
for alexnet_test_dataset_df_infos_keys in alexnet_test_dataset_df_infos.keys():
  df = alexnet_test_dataset_df_infos[alexnet_test_dataset_df_infos_keys].copy()
  df_without_diagnose = df.drop('Diagnose', axis=1)
  y_test = df['Diagnose'].values
  dataset_test = get_samples_as_dataloader(df_without_diagnose.values, y_test)

  classes = np.unique(y_test)
  le = OneHotEncoder()
  classes_idx = le.fit_transform(classes.reshape(-1, 1)).toarray()

  class_dict = { cls : cls_idx for cls, cls_idx in zip(classes, classes_idx)}
  # sorted_class_list = sorted(class_dict.items(), key=lambda kv: (kv[1], kv[0]))
  # sorted_class_labels = [d_key for d_key in dict(sorted_class_list).keys()]
  sorted_class_labels = class_dict

  dataset_test_loader = DataLoader(dataset_test, shuffle=True, batch_size=10 if 'MIX' in alexnet_test_dataset_df_infos_keys else 5)
  alexnet_test_results_infos[alexnet_test_dataset_df_infos_keys] = test_model_without_save_as_image(alexnet_best_model_infos[alexnet_test_dataset_df_infos_keys], f'AlexNet_{alexnet_test_dataset_df_infos_keys}','cpu', dataset_test_loader, sorted_class_labels)


In [ ]:


lstm_v3_test_results_infos = {}
print('LSTM V3 Test Result Collectting')
for lstm_v3_procedure_path_infos_key in lstm_v3_procedure_path_infos.keys():

  df = lstm_v3_test_dataset_df_infos[lstm_v3_procedure_path_infos_key].copy()
  df_without_diagnose = df.drop('Diagnose', axis=1)
  y_test = df['Diagnose'].values
  dataset_test = get_samples_as_dataloader(df_without_diagnose.values, y_test, True)

  classes = np.unique(y_test)
  le = OneHotEncoder()
  classes_idx = le.fit_transform(classes.reshape(-1, 1)).toarray()

  class_dict = { cls : cls_idx for cls, cls_idx in zip(classes, classes_idx)}
  # sorted_class_list = sorted(class_dict.items(), key=lambda kv: (kv[1], kv[0]))
  # sorted_class_labels = [d_key for d_key in dict(sorted_class_list).keys()]
  sorted_class_labels = class_dict

  dataset_test_loader = DataLoader(dataset_test, shuffle=True, batch_size=10 if 'MIX' in lstm_v3_procedure_path_infos_key else 5)
  lstm_v3_test_results_infos[lstm_v3_procedure_path_infos_key] = test_model_without_save_as_image(lstm_v3_best_model_infos[lstm_v3_procedure_path_infos_key], f'LSTM_V3_{lstm_v3_procedure_path_infos_key}','cpu', dataset_test_loader, sorted_class_labels)


lstm_v4_test_results_infos = {}
print('LSTM V4 Test Result Collectting')
for lstm_v4_procedure_path_infos_key in lstm_v4_procedure_path_infos.keys():

   df = lstm_v4_test_dataset_df_infos[lstm_v4_procedure_path_infos_key].copy()
   df_without_diagnose = df.drop('Diagnose', axis=1)
   y_test = df['Diagnose'].values
   dataset_test = get_samples_as_dataloader(df_without_diagnose.values, y_test, True)

   classes = np.unique(y_test)
   le = OneHotEncoder()
   classes_idx = le.fit_transform(classes.reshape(-1, 1)).toarray()
   class_dict = { cls : cls_idx for cls, cls_idx in zip(classes, classes_idx)}
   # sorted_class_list = sorted(class_dict.items(), key=lambda kv: (kv[1], kv[0]))
   # sorted_class_labels = [d_key for d_key in dict(sorted_class_list).keys()]
   sorted_class_labels = class_dict

   dataset_test_loader = DataLoader(dataset_test, shuffle=True, batch_size=10 if 'MIX' in lstm_v4_procedure_path_infos_key else 5)
   lstm_v4_test_results_infos[lstm_v4_procedure_path_infos_key] = test_model_without_save_as_image(lstm_v4_best_model_infos[lstm_v4_procedure_path_infos_key], f'LSTM_V4_{lstm_v4_procedure_path_infos_key}','cpu', dataset_test_loader, sorted_class_labels)



In [ ]:
fig, axes = plt.subplots(6, 6, figsize=(30, 30))

procedure_list = ['BLNC--ONL_ORG', \
                  'NON_BLNC--ONL_ORG', \
                  'BLNC--ORG_AUG_SEP',\
                  'NON_BLNC--ORG_AUG_SEP',\
                  'BLNC--ORG_AUG_MIX', \
                  'NON_BLNC--ORG_AUG_MIX']

axes_idx = 0

for row_idx in range(6):

  result_components = alexnet_test_results_infos[f'{procedure_list[row_idx]}--ALL']
  df_cm = alexnet_test_results_infos[f'{procedure_list[row_idx]}--ALL'][0].copy()
  sns.heatmap(df_cm, annot=True, fmt='g', ax=axes[row_idx][0])
  procedure_components = procedure_list[row_idx].split('--')
  axes_title = f'AlexNet {procedure_components[0]}, {procedure_components[1]}, ALL'
  axes[row_idx][0].set_title(axes_title)
  print(axes_title)
  print('Correct Prediction: {:d}  Total Samples: {:d}'.format(result_components[1], result_components[2]))
  print('Test Accuracy = {:f}'.format(result_components[3]))
  print('Test F1 Score = {:f}'.format(result_components[4]))
  print('Test Precision = {:f}'.format(result_components[5]))
  print('Test Recal = {:f}'.format(result_components[6]))

  result_components = alexnet_test_results_infos[f'{procedure_list[row_idx]}--TD']
  df_cm = alexnet_test_results_infos[f'{procedure_list[row_idx]}--TD'][0].copy()
  sns.heatmap(df_cm, annot=True, fmt='g', ax=axes[row_idx][1])
  procedure_components = procedure_list[row_idx].split('--')
  axes_title = f'AlexNet {procedure_components[0]}, {procedure_components[1]}, TD'
  axes[row_idx][1].set_title(axes_title)
  print(axes_title)
  print('Correct Prediction: {:d}  Total Samples: {:d}'.format(result_components[1], result_components[2]))
  print('Test Accuracy = {:f}'.format(result_components[3]))
  print('Test F1 Score = {:f}'.format(result_components[4]))
  print('Test Precision = {:f}'.format(result_components[5]))
  print('Test Recal = {:f}'.format(result_components[6]))

  result_components = lstm_test_results_infos[f'{procedure_list[row_idx]}--ALL']
  df_cm = lstm_test_results_infos[f'{procedure_list[row_idx]}--ALL'][0].copy()
  sns.heatmap(df_cm, annot=True, fmt='g', ax=axes[row_idx][2])
  procedure_components = procedure_list[row_idx].split('--')
  axes_title = f'LSTM {procedure_components[0]}, {procedure_components[1]}, ALL'
  axes[row_idx][2].set_title(axes_title)
  print(axes_title)
  print('Correct Prediction: {:d}  Total Samples: {:d}'.format(result_components[1], result_components[2]))
  print('Test Accuracy = {:f}'.format(result_components[3]))
  print('Test F1 Score = {:f}'.format(result_components[4]))
  print('Test Precision = {:f}'.format(result_components[5]))
  print('Test Recal = {:f}'.format(result_components[6]))


  result_components = lstm_test_results_infos[f'{procedure_list[row_idx]}--TD']
  df_cm = lstm_test_results_infos[f'{procedure_list[row_idx]}--TD'][0].copy()
  sns.heatmap(df_cm, annot=True, fmt='g', ax=axes[row_idx][3])
  procedure_components = procedure_list[row_idx].split('--')
  axes_title = f'LSTM {procedure_components[0]}, {procedure_components[1]}, TD'
  axes[row_idx][3].set_title(axes_title)
  print(axes_title)
  print('Correct Prediction: {:d}  Total Samples: {:d}'.format(result_components[1], result_components[2]))
  print('Test Accuracy = {:f}'.format(result_components[3]))
  print('Test F1 Score = {:f}'.format(result_components[4]))
  print('Test Precision = {:f}'.format(result_components[5]))
  print('Test Recal = {:f}'.format(result_components[6]))


  result_components = lstm_v2_test_results_infos[f'{procedure_list[row_idx]}--ALL']
  df_cm = lstm_v2_test_results_infos[f'{procedure_list[row_idx]}--ALL'][0].copy()
  sns.heatmap(df_cm, annot=True, fmt='g', ax=axes[row_idx][4])
  procedure_components = procedure_list[row_idx].split('--')
  axes_title = f'LSTM V2 {procedure_components[0]}, {procedure_components[1]}, ALL'
  axes[row_idx][4].set_title(axes_title)
  print(axes_title)
  print('Correct Prediction: {:d}  Total Samples: {:d}'.format(result_components[1], result_components[2]))
  print('Test Accuracy = {:f}'.format(result_components[3]))
  print('Test F1 Score = {:f}'.format(result_components[4]))
  print('Test Precision = {:f}'.format(result_components[5]))
  print('Test Recal = {:f}'.format(result_components[6]))


  result_components = lstm_v2_test_results_infos[f'{procedure_list[row_idx]}--TD']
  df_cm = lstm_v2_test_results_infos[f'{procedure_list[row_idx]}--TD'][0].copy()
  sns.heatmap(df_cm, annot=True, fmt='g', ax=axes[row_idx][5])
  procedure_components = procedure_list[row_idx].split('--')
  axes_title = f'LSTM V2  {procedure_components[0]}, {procedure_components[1]}, TD'
  axes[row_idx][5].set_title(axes_title)
  print(axes_title)
  print('Correct Prediction: {:d}  Total Samples: {:d}'.format(result_components[1], result_components[2]))
  print('Test Accuracy = {:f}'.format(result_components[3]))
  print('Test F1 Score = {:f}'.format(result_components[4]))
  print('Test Precision = {:f}'.format(result_components[5]))
  print('Test Recal = {:f}'.format(result_components[6]))



In [ ]:
fig, axes = plt.subplots(6, 2, figsize=(30, 30))

procedure_list = ['BLNC--ONL_ORG--ALL', \
                  'BLNC--ORG_AUG_SEP--ALL',\
                  'NON_BLNC--ORG_AUG_SEP--ALL',\
                  'BLNC--ORG_AUG_MIX--ALL', \
                  'NON_BLNC--ORG_AUG_MIX--ALL']

axes_idx = 0

for row_idx in range(6):

  if row_idx < 5:
    result_components = lstm_v3_test_results_infos[f'{procedure_list[row_idx]}--NON_RAW']
    df_cm = lstm_v3_test_results_infos[f'{procedure_list[row_idx]}--NON_RAW'][0].copy()
    sns.heatmap(df_cm, annot=True, fmt='g', ax=axes[row_idx][0])
    procedure_components = procedure_list[row_idx].split('--')
    axes_title = f'LSTM V3 {procedure_components[0]}, {procedure_components[1]}, {procedure_components[2]}, NON RAW'
    axes[row_idx][0].set_title(axes_title)
    print(axes_title)
    print('Correct Prediction: {:d}  Total Samples: {:d}'.format(result_components[1], result_components[2]))
    print('Test Accuracy = {:f}'.format(result_components[3]))
    print('Test F1 Score = {:f}'.format(result_components[4]))
    print('Test Precision = {:f}'.format(result_components[5]))
    print('Test Recal = {:f}'.format(result_components[6]))


    result_components = lstm_v4_test_results_infos[procedure_list[row_idx]]
    df_cm = lstm_v4_test_results_infos[procedure_list[row_idx]][0].copy()
    sns.heatmap(df_cm, annot=True, fmt='g', ax=axes[row_idx][1])
    procedure_components = procedure_list[row_idx].split('--')
    axes_title = f'LSTM V4 {procedure_components[0]}, {procedure_components[1]}, {procedure_components[2]}, TD'
    axes[row_idx][1].set_title(axes_title)
    print(axes_title)
    print('Correct Prediction: {:d}  Total Samples: {:d}'.format(result_components[1], result_components[2]))
    print('Test Accuracy = {:f}'.format(result_components[3]))
    print('Test F1 Score = {:f}'.format(result_components[4]))
    print('Test Precision = {:f}'.format(result_components[5]))
    print('Test Recal = {:f}'.format(result_components[6]))

  else:

    result_components = lstm_v3_test_results_infos[f'{procedure_list[3]}--RAW']
    df_cm = lstm_v3_test_results_infos[f'{procedure_list[3]}--RAW'][0].copy()
    sns.heatmap(df_cm, annot=True, fmt='g', ax=axes[row_idx][0])
    procedure_components = procedure_list[3].split('--')
    axes_title = f'LSTM V3 {procedure_components[0]}, {procedure_components[1]}, {procedure_components[2]}, RAW'
    axes[row_idx][0].set_title(axes_title)
    print(axes_title)
    print('Correct Prediction: {:d}  Total Samples: {:d}'.format(result_components[1], result_components[2]))
    print('Test Accuracy = {:f}'.format(result_components[3]))
    print('Test F1 Score = {:f}'.format(result_components[4]))
    print('Test Precision = {:f}'.format(result_components[5]))
    print('Test Recal = {:f}'.format(result_components[6]))


